In [45]:
import torch
from tqdm.auto import tqdm

from models.vae_flow import *
from models.flowmatch import *

import open3d as o3d

In [46]:
device = "cuda" if torch.cuda.is_available() else "cpu"
objects = ["airplane", "bag", "table", "car"]

def normalize_point_clouds(pcs):
    #normalize via shape_bbox
    for i in tqdm(range(pcs.size(0))):
        pc = pcs[i]
        pc_max, _ = pc.max(dim=0, keepdim=True) # (1, 3)
        pc_min, _ = pc.min(dim=0, keepdim=True) # (1, 3)
        shift = ((pc_min + pc_max) / 2).view(1, 3)
        scale = (pc_max - pc_min).max().reshape(1, 1) / 2
        pc = (pc - shift) / scale
        pcs[i] = pc
    return pcs

In [47]:
#FlowVAE original paper ddpm
flow_vae_pcs = {}
for obj in objects:
    ckpt_file_dir = f"pretrained/ckpt_gen_{obj}.pt"

    if device == "cuda":
        ckpt = torch.load(ckpt_file_dir)
    else:
        ckpt = torch.load(ckpt_file_dir, map_location="cpu")

    model = FlowVAE(ckpt['args']).to(device)

    with torch.no_grad():
        z = torch.randn([1, ckpt['args'].latent_dim]).to(device)
        x = model.sample(z, 2048, flexibility=ckpt['args'].flexibility)

    x = normalize_point_clouds(x)
    flow_vae_pcs[obj] = x

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [48]:
obj_name = "bag"
test = flow_vae_pcs[obj_name].detach().cpu().numpy()[0]

spheres = None
r = 0.015
for p in test:
    point = np.expand_dims(p, 0).T
    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=r)
    sphere.translate(point)

    if spheres == None:
        spheres = sphere
    else:
        spheres += sphere

spheres.paint_uniform_color([.33, 0.53, 0.62])

o3d.visualization.draw_geometries(
    [spheres],
    lookat=spheres.get_center(),
)

In [49]:
#fm original paper ddpm
fm_pcs = {}
for obj in objects:
    ckpt_file_dir = f"pretrained/ckpt_fm_{obj}.pt"

    if device == "cuda":
        ckpt = torch.load(ckpt_file_dir)
    else:
        ckpt = torch.load(ckpt_file_dir, map_location="cpu")

    model = FlowMatch(ckpt['args']).to(device)

    with torch.no_grad():
        z = torch.randn([1, ckpt['args'].latent_dim]).to(device)
        x = model.sample(z, 2048, flexibility=ckpt['args'].flexibility)

    x = normalize_point_clouds(x)
    fm_pcs[obj] = x

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [51]:
obj_name = "airplane"
test = fm_pcs[obj_name].detach().cpu().numpy()[0]

spheres = None
r = 0.015
for p in test:
    point = np.expand_dims(p, 0).T
    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=r)
    sphere.translate(point)

    if spheres == None:
        spheres = sphere
    else:
        spheres += sphere

spheres.paint_uniform_color([.33, 0.53, 0.62])

o3d.visualization.draw_geometries(
    [spheres],
    lookat=spheres.get_center(),
)